In [1]:
import pandas as pd

books = pd.read_csv('dataset/books_cleaned.csv')

In [2]:
books['categories'].nunique()

479

In [3]:
books["categories"].value_counts().reset_index().query("count > 50")

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
5,Religion,117
6,Philosophy,117
7,Comics & Graphic Novels,116
8,Drama,86
9,Juvenile Nonfiction,57


In [4]:
category_mapping = {'Fiction' : "Fiction",
 'Juvenile Fiction': "Fiction",
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Juvenile Nonfiction': "Nonfiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

books["simple_categories"] = books["categories"].map(category_mapping)

In [5]:
books['simple_categories'].nunique()

2

In [6]:
books[~books['simple_categories'].isna()].shape

(3743, 14)

In [7]:
simplified_books = books[~books['simple_categories'].isna()]

In [8]:
simplified_books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
30,9780006646006,000664600X,Ocean Star Express,Mark Haddon;Peter Sutton,Juvenile Fiction,http://books.google.com/books/content?id=I2QZA...,Joe and his parents are enjoying a summer holi...,2002.0,3.50,32.0,1.0,Ocean Star Express,9780006646006 Joe and his parents are enjoying...,Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [9]:
from transformers import pipeline

categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification",
                model="facebook/bart-large-mnli",
                device="cpu")

Device set to use cpu


In [10]:
sequence = simplified_books.loc[simplified_books['simple_categories']== "Fiction", "description"].reset_index(drop=True)

In [11]:
pipe(sequence, categories)


{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [12]:
import numpy as np

max_index = np.argmax(pipe(sequence, categories)["scores"])
max_label = pipe(sequence, categories)['labels'][max_index]
max_label

'Fiction'

In [13]:
def generate_predictions(sequence,categories):
    predictions = pipe(sequence,categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [14]:
from tqdm import tqdm

actual_cat = []
predicted_cat = []

for i in tqdm(range(0,301)):
    sequence = simplified_books.loc[simplified_books['simple_categories']== "Fiction", "description"].reset_index(drop=True)
    predicted_cat += [generate_predictions(sequence, categories)] 
    actual_cat += ["Fiction"]

100%|██████████| 301/301 [11:28<00:00,  2.29s/it]


In [15]:
for i in tqdm(range(0,301)):
    sequence = simplified_books.loc[simplified_books['simple_categories']== "Nonfiction", "description"].reset_index(drop=True)
    predicted_cat += [generate_predictions(sequence, categories)] 
    actual_cat += ["Nonfiction"]

100%|██████████| 301/301 [07:45<00:00,  1.55s/it]


In [16]:
predictions = pd.DataFrame({"actual_catefories":actual_cat, "pred_categories":predicted_cat})

In [17]:
predictions

,actual_catefories,pred_categories
0,Fiction,Fiction
1,Fiction,Fiction
2,Fiction,Fiction
3,Fiction,Fiction
4,Fiction,Fiction
...,...,...
597,Nonfiction,Nonfiction
598,Nonfiction,Nonfiction
599,Nonfiction,Nonfiction
600,Nonfiction,Nonfiction


In [18]:
predictions["correct_pred"]= (np.where(predictions["actual_catefories"]==predictions["pred_categories"],True, False))

In [19]:
predictions["correct_pred"].sum()/len(predictions)

1.0

In [20]:
isbns =[]
predicted_cat =[]

missing_cat = books.loc[books["simple_categories"].isna(),["isbn13","description"]].reset_index(drop=True)

In [21]:
missing_cat

,isbn13,description
0,9780002261982,A new 'Christie for Christmas' -- a full-lengt...
1,9780006280897,Lewis' work on the nature of love divides love...
2,9780006280934,"""In The Problem of Pain, C.S. Lewis, one of th..."
3,9780006380832,Until Vasco da Gama discovered the sea-route t...
4,9780006470229,A new-cover reissue of the fourth book in the ...
...,...,...
1449,9788125026600,Not only does Nietzsche for Beginners delve in...
1450,9788171565641,"Forster's lively, informed originality and wit..."
1451,9788172235222,On A Train Journey Home To North India After L...
1452,9788173031014,This book tells the tale of a man who goes on ...


In [22]:
for i in tqdm(range(0,len(missing_cat))):
    sequence = missing_cat["description"][i]
    predicted_cat +=[generate_predictions(sequence, categories)]
    isbns += [missing_cat["isbn13"][i]]

100%|██████████| 1454/1454 [25:44<00:00,  1.06s/it]


In [24]:
missing_pred = pd.DataFrame({"isbn13":isbns, "pred_cat":predicted_cat})

In [25]:
missing_pred

,isbn13,pred_cat
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [27]:
books = pd.merge(simplified_books, missing_pred, on="isbn13",how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["pred_cat"], books["simple_categories"])
books = books.drop(columns = ["pred_cat"])

In [28]:
books['simple_categories'].unique()

array(['Fiction', 'Nonfiction'], dtype=object)

In [29]:
books.to_csv("dataset/simple_cat_books.csv", index=False)